In [3]:
import os
print(os.getcwd())


C:\Users\gbrea\PY\mi_entorno


In [ ]:
!pip install dash pandas openpyxl plotly


In [5]:
import dash
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# Cargar datos
df = pd.read_excel("Datos-Coupa.xlsx")  # Ajusta la ruta a tu archivo

In [6]:
# Imprimir los nombres de columnas para verificación
print(df.columns.tolist())


['Código\nCIUO-88', 'Ocupación', 'clasificacion_refinada', 'Descripción del Grupo', 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 'Años de variación 2008-2015', 'Años de variación 2016-2019', 'Años de variación 2020-2023', 'Años de variación 2008-2023', 'Tasa de variación simple del empleo 2008-2015', 'Tasa de variación simple del empleo 2016-2019', 'Tasa de variación simple del empleo 2020-2023', 'Tasa de variación simple del empleo 2008-2023', 'Tasa de variación anual ponderada del empleo 2008-2015', 'Tasa de variación anual ponderada del empleo 2016-2019', 'Tasa de variación anual ponderada del empleo 2020-2023', 'Tasa de variación anual ponderada del empleo 2008-2023', 'Tasa esperada 2020-2023', 'Media 2020-2023', 'Desvio estándar 2020-2023', 'CV 2020-2023', 'Tipo de crecimiento', 'ICDE', 'ICDE - Categoria', 'Tamaño ocupaciones', 'Percentil 75 Media 2020-2023', 'Percentil 25 Media 2020-2023', 'Percentil 75 de crecimiento ICDE', 'Per

In [7]:
# Inicializar la aplicación Dash
app = Dash(__name__)

# Asegurarse de que las columnas de años son strings para facilitar la manipulación
df.columns = df.columns.map(str)

# Lista de años como strings para los sliders
years = [str(year) for year in range(2008, 2024)]

# Definir la disposición de la aplicación
app.layout = html.Div([
    html.H1("Dashboard de Ocupaciones"),
    dcc.Dropdown(
        id='occupation-dropdown',
        options=[{'label': i, 'value': i} for i in df['Ocupación'].unique()],
        value=df['Ocupación'].unique()[0]
    ),
    dcc.RangeSlider(
        id='year-range-slider',
        min=2008,
        max=2023,
        step=1,
        value=[2008, 2023],
        marks={year: year for year in years}
    ),
    dcc.Graph(id='employment-graph'),
    dcc.Graph(id='growth-graph'),
])

# Definir la interactividad
@app.callback(
    [Output('employment-graph', 'figure'), Output('growth-graph', 'figure')],
    [Input('occupation-dropdown', 'value'), Input('year-range-slider', 'value')]
)
def update_graphs(selected_occupation, year_range):
    # Filtrar por ocupación
    filtered_df = df[df['Ocupación'] == selected_occupation]
    
    # Transformar datos a formato long
    melted_df = filtered_df.melt(id_vars=['Ocupación', 'clasificacion_refinada', 'Descripción del Grupo'], 
                                 value_vars=[year for year in years if int(year) >= year_range[0] and int(year) <= year_range[1]],
                                 var_name='Año', value_name='Empleo')

    # Crear gráficos
    fig_employment = px.line(melted_df, x='Año', y='Empleo', title='Empleo a lo largo del tiempo')
    fig_growth = px.bar(melted_df, x='Año', y='Empleo', title='Crecimiento por Año')

    return fig_employment, fig_growth

# Correr la aplicación en modo adecuado para notebooks
if __name__ == '__main__':
    app.run_server(mode='external')  # Cambiado de 'inline' a 'external' para adaptarse a cambios recientes

In [8]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.
